In [68]:
%matplotlib inline
from __future__ import division, print_function
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import ZeroPadding2D
from keras.layers import MaxPooling2D
import numpy as np
from keras.preprocessing import image 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from utils import plots

In [37]:
idg = image.ImageDataGenerator()
train_generator = idg.flow_from_directory('/mnt/data/dogscats/train/', batch_size=1)

Found 23000 images belonging to 2 classes.


In [11]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))

In [14]:
def vgg_preprocess():
    x = x - vgg_mean

array([[[ 123.68000031]],

       [[ 116.77899933]],

       [[ 103.93900299]]], dtype=float32)

In [ ]:
def ConvBlock(model, layers, filters):
    for i in range(0, layers):
        model.add(ZeroPadding2D((1, 1))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='valid'))

In [7]:
model = Sequential()


ValueError: The first layer in a Sequential model must get an `input_shape` or `batch_input_shape` argument.